In [10]:
import json, math, itertools, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# AWS Braket SDK
from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.devices import LocalSimulator

In [11]:
# Carregar dataset reduzido já com labels
df = pd.read_csv("metal_features_reduzido_com_y.csv")

features = ["Mendeleev", "Tmelt_K", "rcov_A", "d_band_center"]
X_raw = df[features].to_numpy(dtype=float)   # <- esta é a matriz bruta (antes de normalizar)
y     = df["y"].to_numpy(dtype=int)
metals = df["metal"].astype(str).tolist()

print("X_raw shape:", X_raw.shape)
print("y shape:", y.shape)


X_raw shape: (9, 4)
y shape: (9,)


In [12]:
# %pip install amazon-braket-sdk amazon-braket-default-simulator -q

from braket.aws import AwsDevice, AwsSession
from braket.circuits import Circuit
from braket.devices import LocalSimulator

# Sessão na região (ajuste se usar outra)
session = AwsSession()  # ou us-west-1, eu-west-2 etc.

# SV1 (state-vector simulator gerenciado)
sv1 = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1", aws_session=session)

# Outros simuladores gerenciados:
dm1 = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1", aws_session=session)  # density matrix
tn1 = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/tn1", aws_session=session)  # tensor network

# Ex.: listar QPUs disponíveis na região da sessão (para pegar o ARN certo)
from braket.aws import AwsDeviceType, AwsQuantumTaskBatch
qpus = [d for d in AwsDevice.get_devices(aws_session=session) if d.type == AwsDeviceType.QPU]
for d in qpus:
    print(d.name, "->", d.arn)


Ankaa-3 -> arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3
Aquila -> arn:aws:braket:us-east-1::device/qpu/quera/Aquila
Aria 1 -> arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1
Aria 2 -> arn:aws:braket:us-east-1::device/qpu/ionq/Aria-2
Emerald -> arn:aws:braket:eu-north-1::device/qpu/iqm/Emerald
Forte 1 -> arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1
Forte Enterprise 1 -> arn:aws:braket:us-east-1::device/qpu/ionq/Forte-Enterprise-1
Garnet -> arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet


In [13]:
import numpy as np
from braket.circuits import Circuit

def feature_map_ry_cz(x, layers=2):
    # x: 4 features em [0,1]
    c = Circuit()
    for _ in range(layers):
        for q, xi in enumerate(x):
            c.ry(q, np.pi * xi)
        c.cz(0,1).cz(1,2).cz(2,3)
    return c

def overlap_circuit(x, xp, layers=2):
    Ux  = feature_map_ry_cz(x,  layers)
    Uxp = feature_map_ry_cz(xp, layers)
    return Circuit().add_circuit(Ux.adjoint()).add_circuit(Uxp)


In [5]:
x_demo  = [0.1,0.2,0.3,0.4]
xp_demo = [0.2,0.1,0.4,0.3]
print("=== U(x) ==="); print(feature_map_ry_cz(x_demo, layers=2))
print("\n=== U(x)^† U(x') ==="); print(overlap_circuit(x_demo, xp_demo, layers=2))


=== U(x) ===
T  : │     0      │  1  │     2      │     3      │     4      │  5  │  6  │
      ┌──────────┐       ┌──────────┐                                       
q0 : ─┤ Ry(0.31) ├───●───┤ Ry(0.31) ├───────────────────●───────────────────
      └──────────┘   │   └──────────┘                   │                   
      ┌──────────┐ ┌─┴─┐              ┌──────────┐    ┌─┴─┐                 
q1 : ─┤ Ry(0.63) ├─┤ Z ├──────●───────┤ Ry(0.63) ├────┤ Z ├───────●─────────
      └──────────┘ └───┘      │       └──────────┘    └───┘       │         
      ┌──────────┐          ┌─┴─┐                  ┌──────────┐ ┌─┴─┐       
q2 : ─┤ Ry(0.94) ├──────────┤ Z ├──────────●───────┤ Ry(0.94) ├─┤ Z ├───●───
      └──────────┘          └───┘          │       └──────────┘ └───┘   │   
      ┌──────────┐                       ┌─┴─┐     ┌──────────┐       ┌─┴─┐ 
q3 : ─┤ Ry(1.26) ├───────────────────────┤ Z ├─────┤ Ry(1.26) ├───────┤ Z ├─
      └──────────┘                       └───┘     └──────────┘

In [17]:
def fidelity_any(device, x, xp, layers=2, shots=2000):
    """
    Estima |<Phi(x)|Phi(x')>|^2 como frequência de '0000'.
    Funciona em SV1/DM1/TN1 e em QPUs.
    """
    circ = overlap_circuit(x, xp, layers)
    circ = circ.measure_all()            # mede todos os qubits
    task = device.run(circ, shots=shots)
    counts = task.result().measurement_counts  # dict: 'bitstring' -> contagem
    return counts.get('0000', 0) / shots



In [15]:
def fidelity_qpu(device, x, xp, layers=2, shots=2000):
    circ = overlap_circuit(x, xp, layers)
    circ = circ.measure_all()  # mede todos os qubits
    task = device.run(circ, shots=shots)
    counts = task.result().measurement_counts  # dict: bitstring -> contagem
    # bitstring de 4 qubits no Braket é '0000' para |0...0> (little-endian por default)
    p0000 = counts.get('0000', 0) / shots
    return float(p0000)


In [16]:
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# escolha do device:
device = sv1          # simulador gerenciado (exato, rápido)
# device = dm1        # também ok com shots=0
# device = tn1        # usar versão "qpu" (shots > 0)
# device = AwsDevice("arn:.../qpu/ionq/Aria-2", aws_session=session)  # exemplo: QPU -> usar versão "qpu"

# Se for SV1/DM1 -> use fidelity_simulator; se for TN1/QPU -> fidelity_qpu
use_qpu_mode = (device.properties.dict().get("deviceType") == "QPU") or ("tn1" in device.arn)

def fidelity_dispatch(x, xp, layers=2):
    if use_qpu_mode:
        return fidelity_qpu(device, x, xp, layers=layers, shots=2000)
    else:
        return fidelity_simulator(device, x, xp, layers=layers)

# … dentro do LOO:
loo = LeaveOneOut()
y_true, y_pred = [], []
for tr, te in loo.split(X_raw):
    scaler = MinMaxScaler().fit(X_raw[tr])
    Xtr, Xte = scaler.transform(X_raw[tr]), scaler.transform(X_raw[te])

    n = len(Xtr)
    Ktr = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            Ktr[i,j] = fidelity_dispatch(Xtr[i], Xtr[j], layers=2)

    Kte = np.zeros((1,n))
    for j in range(n):
        Kte[0,j] = fidelity_dispatch(Xte[0], Xtr[j], layers=2)

    clf = SVC(kernel="precomputed", C=1.0)
    clf.fit(Ktr, y[tr])
    y_pred.append(clf.predict(Kte)[0])
    y_true.append(y[te][0])

print("QSVM (device =", device.name, ") LOO acc =", accuracy_score(y_true, y_pred))


ValidationException: An error occurred (ValidationException) when calling the CreateQuantumTask operation: [line 31] for result type probability, the shot should be between 1 and 100000 on the requested device